In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import bokeh
import matplotlib.pyplot as plt
import numpy as np
import os
import urllib

In [8]:
import numpy as np
import sys
import h5py
#sys.path.append('/reg/neh/home/tdd14/tmo_repo') #this is a temporary home for the general TMO analysis code
sys.path.append('/cds/home/p/paris/reconVMI')
import analyzers_anna as az # this raises a few runtime errors which require some updating

import psana as ps
from matplotlib import pyplot as plt
from tqdm import tqdm
import matplotlib.colors as colors
from scipy.ndimage import gaussian_filter as gf
from scipy.ndimage import median_filter as mf
from scipy import optimize

from matplotlib import cm
from matplotlib.colors import ListedColormap
from scipy.optimize import minimize
from scipy.signal import find_peaks
import scipy
#from scipy.ndimage import gaussian_filter
import sys
sys.path.append('/reg/neh/home/tdd14/modules/cart2pol/cart2pol')
import cart2pol
import scipy.misc as scmis
from scipy.optimize import minimize
from scipy.optimize import curve_fit
import scipy.special as sp
from datetime import datetime
import cv2
from recon_funcs import *


In [4]:
def make_MM(x0):
    xshift, yshift =  x0[8], x0[9] 

    # Coordinates that you want to Perspective Transform
    pts1 = np.float32([[0,0],\
                       [0,1024],\
                       [1024,0],\
                       [1024, 1024]])


    # Size of the Transformed Image
    pts2 = np.float32([[0+xshift-x0[0],0 + yshift-x0[1]],\
                       [0+xshift-x0[2],1024 + yshift-x0[3]],\
                       [1024+xshift-x0[4],0 + yshift-x0[5]],\
                       [1024+xshift-x0[6],1024 + yshift-x0[7]]])

    MM = cv2.getPerspectiveTransform(pts1,pts2)
    return MM


MM = make_MM([-17.998,-2.74, 17.9, -17.39, 17.86, 8.52, -3.85, 17.9, 11.47, 47.86])

X,Y=np.meshgrid(np.arange(1024),np.arange(1024))
rs=np.sqrt((X-512)**2+(Y-512)**2)
maskhm=np.zeros((1024,1024))
maskhm[(rs>120) & (rs<420)]=1
X,Y=np.meshgrid(np.arange(1024),np.arange(1024))
rs=np.sqrt((X-512)**2+(Y-512)**2)
maskhmb=np.zeros((1024,1024))
maskhmb[(rs>120) & (rs<450)]=1
nmask = np.reshape(rebin(rebin(maskhm,4),4),[64**2,])


x,y, = 530,512
X,Y=np.meshgrid(np.arange(1024),np.arange(1024))
rs=np.sqrt((X-x)**2+(Y-y)**2)
mask=np.zeros((1024,1024))
mask[rs>1*48]=1

nmask = np.reshape(rebin(rebin(maskhm,4),4),[64**2,]) #rebin later
nnmask = (rebin(np.reshape(nmask, [64,64]))).flatten()
nnmask = rebin(rebin(maskhm,4),4)
#nnmask = (rebin(np.reshape(nmask, [64,64])*(inbg))).flatten()
nnmask[nnmask<0.5] = 0
nnmask[nnmask>=0.5] = 1

maskscale = np.zeros((1024,1024))
X,Y=np.meshgrid(np.arange(1024),np.arange(1024))
rs=np.sqrt((X-512)**2+(Y-512)**2)
maskscale [(rs>120) & (rs<270)]=1

def make_quads(x0):
    quads=np.ones((1024,1024))
    quads[:512,:512]= x0[0]
    quads[:512,512:]= x0[1]
    quads[512:,:512]= x0[2]
    quads[512:,512:]= x0[3]
    return quads

In [5]:
import matplotlib
from matplotlib import  cm
colormap =cm.get_cmap("viridis") #choose any matplotlib colormap here
bokehpalette = [matplotlib.colors.rgb2hex(m) for m in colormap(np.arange(colormap.N))]

In [10]:
exp = "tmox51020" #"tmoly9120" #'tmox45719'
gas_jet = 1
runnum = 131 #127 #122 #121 #112 #105 #76 #73 LY91, 141
gmd_min = 0.12 #0.33
gmd_max = 0.2
timestamp_min = 0

thresh = 32
mfn = 25 
gfn = 15

gfsig = 0.985 
gfcart = 0.25 
mfcart = 1

In [7]:
MM = make_MM([-17.998,-2.74, 17.9, -17.39, 17.86, 8.52, -3.85, 17.9, 11.47, 47.86])

bgdat = np.load('/cds/home/p/paris/cVMI/bgdat_run_131_Aug21.npy')
bins = bgdat['bins_maxim']    
#binvar = maxim #detsum
bg_im = bgdat['bg_im_maxim'] #bgdat['bg_im']

In [12]:
cVMIs = []
gds = []
tts = []

ds=ps.DataSource(exp = 'tmox51020', run= runnum)

run = next(ds.runs())

opal1 = run.Detector('tmo_opal1') # OPAL camera looking at the holey mirror
gmd = run.Detector("gmd") # gas intensity detector 
timing = run.Detector('timing') # this gives access to evr codes


if hasattr(run, 'epicsinfo'):
    epics_strs = [item[0] for item in run.epicsinfo.keys()][1:] # first one is weird
    epics_detectors = [run.Detector(item) for item in epics_strs]    

for nevent, event in enumerate(run.events()):

    data = {'epics_'+epic_str: epic_det(event) for epic_str, epic_det in zip(epics_strs, epics_detectors)}        
    if any(type(val) not in [int, float] for val in data.values()):
        print("Bad EPICS: %d" % nevent)
        #Nbad += 1
        continue
    
    opal1im_i=np.copy(opal1.raw.raw(event))
    gd_i=gmd.raw.energy(event)
    evr=timing.raw.eventcodes(event)

    # identify relevant event codes
    gas=evr[70] # did the gas jet fire
    ir_timed=evr[67] # was the ir laser timed
    ir_misttimed=evr[68] # was the ir laser mis-timed
    
    if event.timestamp <= timestamp_min: continue
    
    if opal1im_i is None or gd_i is None or gd_i<gmd_min  or gd_i> gmd_max or (gas != gas_jet) or (ir_timed != 1): continue

    im_look=np.copy(opal1.raw.raw(event))  #opal1im_i=np.copy(opal1.raw.image(event)) #doesn't work since update
    im1=im_look

    ims = np.rot90(im1*mask)
    ims[ims<30] = 0
    
    binvar = np.max(rebin(rebin(ims,4),4))
    idx = np.max((np.argmax(bins-binvar>0)-1, 0))
    idx = np.array([0,1])
    #idx = np.arange(idx-1,idx+1)
    #idx = np.arange(1,13)
    idx_r = np.array([1])
    try:
        len(idx_r)
        resid = np.mean(bg_im[idx_r],axis = 0)
    except: 
        resid = bg_im[idx_r]
    
    try:
        len(idx)
        qqs = np.mean(bgdat['quads'][idx],axis = 0)
    except: 
        qqs = bgdat['quads'][idx]

    
    
    if 0 in [np.sum(qqss) for qqss in bgdat['quads'][idx]]: continue #need to fix this
    
    #qqs = [0.91      , 0.94795643, 0.94637416, 0.99]
    #qqs0 = [1      ,1, 1.05, 1.1]
    #qqs = qqs * [0.99 ,1, 1.01, 1.01]
    quads = make_quads(qqs)
    ims2 = cv2.warpPerspective(ims*quads,MM,(1024,1024))
    ims2[ims2<0] = 0
    ims2 = gf(ims2, gfn)
    ims2 = rebin(rebin(ims2,4),4)
    polimg = PR.cart2pol(ims2,32,32)
    bb = np.zeros((64,64))
    bb[1:64,1:64]= PR.pol2cart(gf(polimg,[0.3,1.5])) #radial, theta
    ims2 = gf(bb, 0.5)
    #qqs0 = np.mean(bgdat['quads'],axis = 0)
    qqs0 = np.array([0.98849101, 0.99340145, 0.99498136, 1.        ])
    q0 = np.mean(bgdat['quads'][[0,1]],axis = 0)
    #resid = resid*rebin(rebin(cv2.warpPerspective(make_quads(1/qqs0*q0),MM,(1024,1024)),4),4)
    resid = resid*rebin(rebin(cv2.warpPerspective(make_quads(1/qqs0),MM,(1024,1024)),4),4)
    #resid = resid*rebin(rebin(cv2.warpPerspective(make_quads(1/qqs0*qqs0),MM,(1024,1024)),4),4)
    resid = gf(resid, 0.5+5/16)
    resid_scale = resid/np.mean(resid*rebin(rebin(maskscale,4),4))*np.mean(ims2*rebin(rebin(maskscale,4),4))
    #resid_scale[:32,:] = resid[:32,:]/np.mean(resid[:32,:]*rebin(rebin(maskscale,4),4)[:32,:])*np.mean(ims2[:32,:]*rebin(rebin(maskscale,4),4)[:32,:])
    #resid_scale[32:,:] = resid[32:,:]/np.mean(resid[32:,:]*rebin(rebin(maskscale,4),4)[32:,:])*np.mean(ims2[32:,:]*rebin(rebin(maskscale,4),4)[32:,:])
    ims3 = ims2 - resid_scale#*.99
    ims3 = ims3*rebin(rebin(maskhm,4),4)*250/np.max(ims3)
    ims3[ims3<thresh] = 0
     
    cVMIs.append(ims3)
    gds.append(gd_i)
    tts.append(event.timestamp) 
    
    if (len(tts)%10 == 0): print(len(tts))
    #if len(tts)> 1500: break
    if len(tts)> 2000: break

Bad EPICS: 0
Bad EPICS: 1
Bad EPICS: 2
Bad EPICS: 3
Bad EPICS: 4
Bad EPICS: 5
Bad EPICS: 6
Bad EPICS: 7
Bad EPICS: 8
Bad EPICS: 9
Bad EPICS: 10
Bad EPICS: 11
Bad EPICS: 12
Bad EPICS: 13
Bad EPICS: 14
Bad EPICS: 15
Bad EPICS: 16
Bad EPICS: 17
Bad EPICS: 18
Bad EPICS: 19
Bad EPICS: 20
Bad EPICS: 21
Bad EPICS: 22
Bad EPICS: 23
Bad EPICS: 24
Bad EPICS: 25
Bad EPICS: 26
Bad EPICS: 27
Bad EPICS: 28
Bad EPICS: 29
Bad EPICS: 30
Bad EPICS: 31
Bad EPICS: 32
Bad EPICS: 33
Bad EPICS: 34
Bad EPICS: 35
Bad EPICS: 36
Bad EPICS: 37
Bad EPICS: 38
Bad EPICS: 39
Bad EPICS: 40
Bad EPICS: 41
Bad EPICS: 42
Bad EPICS: 43
Bad EPICS: 44
Bad EPICS: 45
Bad EPICS: 46
Bad EPICS: 47
Bad EPICS: 48
Bad EPICS: 49
Bad EPICS: 50
Bad EPICS: 51
Bad EPICS: 52
Bad EPICS: 53
Bad EPICS: 54
Bad EPICS: 55
Bad EPICS: 56
Bad EPICS: 57
Bad EPICS: 58
Bad EPICS: 59
Bad EPICS: 60
Bad EPICS: 61
Bad EPICS: 62
Bad EPICS: 63
Bad EPICS: 64
Bad EPICS: 65
Bad EPICS: 66
Bad EPICS: 67
Bad EPICS: 68
Bad EPICS: 69
Bad EPICS: 70
Bad EPICS: 71
Ba

NameError: name 'PR' is not defined

In [ ]:
b = {}
ids  = {}
gmds = {}
for i in np.arange(2500,5313): #range(2500): #len(cVMIs)):
    b[str(i-2500)] = cVMIs[i]
    ids[str(i-2500)] = [tts[i]]
    gmds[str(i-2500)] = [gds[i]]

In [ ]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Slider, ColumnDataSource, CustomJS, PreText, Button, Circle
from bokeh.layouts import column, row


ims = ColumnDataSource(data = b)
im = ColumnDataSource(data=dict(image = [b[str(1)]]))
IDS = ColumnDataSource(data = ids)
GDS = ColumnDataSource(data = gmds)
p = figure()
p.x_range.range_padding = p.y_range.range_padding = 0

# must give a vector of image data for image parameter
p.image('image', source=im, x=0, y=0, dw=64, dh=64, palette=bokehpalette)

circdeets = ColumnDataSource(dict(x=[32], y=[32], radius=[20.75]))
circ = Circle(x="x", y="y", radius="radius", line_color="white", line_width=3, line_alpha = 0.85, fill_alpha = 0)
p.add_glyph(circdeets, circ)


deet_text = PreText(text='Timestamps')
deet_text2 = PreText(text='Idxs')
slider = Slider(start=0, end=len(tts), value=300, step=1, title="Data File")
button = Button(label="Next")
buttonB = Button(label="Back")
button2 = Button(label="Grab")
pspacer = PreText(text = '                   ')

ptitle = PreText(text = str(IDS.data[str(slider.value)][0])+', gmd = '+str(GDS.data[str(slider.value)][0])+' mJ')

callback = CustomJS(args=dict(ims = ims, im = im, IDS = IDS, GDS = GDS, s = slider, ptitle = ptitle),
                    code="""

    im.data['image'] = [ims.data[s.value]];
    im.change.emit();
    ptitle.text = IDS.data[s.value.toString()][0].toString()+', gmd = '+GDS.data[s.value.toString()][0].toString()+' mJ';
    s.title = 'File '+IDS.data[s.value.toString()][0].toString()  
   
""")

callbackB = CustomJS(args=dict(button = button, s = slider),
                    code="""

    s.value = s.value + 1;
    
    
   
""")

callbackBB = CustomJS(args=dict(button = button, s = slider),
                    code="""

    s.value = s.value - 1;
    
    
   
""")

callbackB2 = CustomJS(args=dict(button2 = button2, IDS = IDS, s = slider, deets = deet_text, deets2 = deet_text2),
                    code="""

    deets.text = deets.text+', ' + IDS.data[s.value.toString()][0].toString();
    deets2.text = deets2.text+', ' + s.value.toString()
    
   
""")



slider.js_on_change('value', callback)
button2.js_on_click(callbackB2)
button.js_on_click(callbackB)
buttonB.js_on_click(callbackBB)
layout = row(column(row(pspacer,ptitle), p, slider), column(row(column(button,buttonB),button2),column(deet_text,deet_text2)))

#output_file("image.html", title="image.py example")
output_notebook()
show(layout)

In [ ]:
tss =  np.array(tts)[[594, 774, 773, 775, 831, 845, 846, 847, 1424, 1749, 1750, 1778, 1867, 2060, 2062, 2065, 2131, 2132, 2211, 2212,2804, 2806, 3169, 3378, 3380, 3415, 3417, 3523, 3524, 3525, 3580]]
len(tss)

In [ ]:
fname = 'streaked_tts_run'+str(runnum)+'_Jul23.npy'
fname

In [ ]:
tss